# Ensemble Model - Combining U-Net and Mask R-CNN

This notebook implements an ensemble approach that combines predictions from both U-Net and Mask R-CNN models for improved traffic sign detection.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")


In [ ]:
DATA_PATH = "../data/car"
UNET_MODEL_PATH = "../data/models/full_dataset_model_traffic_sign_unet.h5"
MASKRCNN_MODEL_PATH = "final_maskrcnn_model.h5"
RESULTS_PATH = "../data/results"

UNET_IMG_SIZE = (256, 256)
MASKRCNN_IMG_SIZE = (128, 128)

os.makedirs(RESULTS_PATH, exist_ok=True)

print(f"Data path exists: {os.path.exists(DATA_PATH)}")
print(f"U-Net model exists: {os.path.exists(UNET_MODEL_PATH)}")
print(f"Mask R-CNN model exists: {os.path.exists(MASKRCNN_MODEL_PATH)}")


def load_yolo_annotations(img_path, img_shape):
    annotation_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    
    if os.path.exists(annotation_path):
        try:
            with open(annotation_path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                data = line.strip().split()
                if len(data) >= 5:
                    x_center, y_center, width, height = map(float, data[1:5])
                    h, w = img_shape[:2]
                    x_center *= w
                    y_center *= h
                    width *= w
                    height *= h
                    x1 = max(0, int(x_center - width/2))
                    y1 = max(0, int(y_center - height/2))
                    x2 = min(w, int(x_center + width/2))
                    y2 = min(h, int(y_center + height/2))
                    if x2 > x1 and y2 > y1:
                        mask[y1:y2, x1:x2] = 1
        except Exception as e:
            print(f"Error processing {annotation_path}: {e}")
    return mask

def load_test_data(data_path, img_size):
    test_path = os.path.join(data_path, 'test', 'images')
    image_files = sorted([f for f in os.listdir(test_path) if f.endswith(('.jpg', '.png'))])
    
    images = []
    masks = []
    filenames = []
    
    for filename in tqdm(image_files, desc="Loading test data"):
        img_path = os.path.join(test_path, filename)
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        original_shape = image.shape
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, img_size)
        image_normalized = image_resized.astype(np.float32) / 255.0
        
        mask = load_yolo_annotations(img_path, original_shape)
        mask_resized = cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST)
        
        images.append(image_normalized)
        masks.append(mask_resized)
        filenames.append(filename)
    
    return np.array(images), np.array(masks), filenames

print("Data loading functions defined.")